In [9]:
import os
import pandas as pd

# 입력 파일 이름
input_file_name = './data/ucosmos-balance.csv'
#input_file_name = './data/ujuno-balance.csv'
#input_file_name = './data/ukava-balance.csv'
#input_file_name = './data/uosmo-balance.csv'

# 데이터셋 로드
df = pd.read_csv(input_file_name)

# Convert _time column to datetime format
df['_time'] = pd.to_datetime(df['_time'])

# '2023-01-27T17:00:00Z' 이전의 데이터 필터링
df = df[df['_time'] >= '2023-01-27T17:00:00Z']

# Create a copy of the dataframe with rows where amountValue is -1 removed
dfm = df[df['amountValue'] != -1]

# Add a new column 'boxplot' with default value 0 to df
df['boxplot'] = 0

# Group both dataframes by moduleName
grouped = df.groupby('moduleName')
grouped_m = dfm.groupby('moduleName')

# Loop through groups in df, but calculate Q1 and Q3 from dfm
for name, group in grouped:
    try:
        Q1 = grouped_m.get_group(name)['amountValue'].quantile(0.25)
        Q3 = grouped_m.get_group(name)['amountValue'].quantile(0.75)
        IQR = Q3 - Q1

        min_value = Q1 - 1.5 * IQR
        max_value = Q3 + 1.5 * IQR

        df.loc[(df['moduleName'] == name) & (df['amountValue'] == -1), 'boxplot'] = -1
        df.loc[(df['moduleName'] == name) & (df['amountValue'] < min_value), 'boxplot'] = 1
        df.loc[(df['moduleName'] == name) & (df['amountValue'] >= min_value) & (df['amountValue'] < Q1), 'boxplot'] = 2
        df.loc[(df['moduleName'] == name) & (df['amountValue'] >= Q1) & (df['amountValue'] < Q3), 'boxplot'] = 3
        df.loc[(df['moduleName'] == name) & (df['amountValue'] >= Q3) & (df['amountValue'] < max_value), 'boxplot'] = 4
        df.loc[(df['moduleName'] == name) & (df['amountValue'] >= max_value), 'boxplot'] = 5
    except KeyError:
        continue

# 파일의 기본 이름(디렉토리 없음)을 가져와서 파일 확장자 제거
base_file_name = os.path.basename(input_file_name)
base_file_name_without_extension = os.path.splitext(base_file_name)[0]

# 결과 파일이 저장될 디렉토리 정의
output_directory = './result-files/'

# 출력 파일 이름 생성
output_file_name = output_directory + base_file_name_without_extension + '.processed.plus.csv'

# 누락된 값의 평균을 수정된 파일 이름의 텍스트 파일로 저장
df.to_csv(output_file_name, header=True)

310129.0 310129.0 0.0
22030092.0 22030092.0 0.0
7310000.0 430503982.5 423193982.5
197333435.0 804767667.0 607434232.0
512074000000.0 890892000000.0 378818000000.0
9354920000000.0 10579100000000.0 1224180000000.0
102291766.0 102291766.0 0.0
4661997001.0 4950926806.0 288929805.0
1300.0 1300.0 0.0
154131000000.0 180613000000.0 26482000000.0
363500000000.0 364545000000.0 1045000000.0


KeyError: 'Bithumb #04'